In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

L'objectif est de prédire si un accident donné est mortel ou non.

In [2]:
label = "mortal"

# Lecture datasets
df1 = pd.read_csv("dataset/usagers-2022.csv", sep=';')
df2 = pd.read_csv("dataset/lieux-2022.csv", sep=';')
df3 = pd.read_csv("dataset/carcteristiques-2022.csv", sep=';')
df4 = pd.read_csv("dataset/vehicules-2022.csv", sep=';')

df4 = df4.drop(columns=['id_vehicule', 'num_veh'])


df = df1.join(df2.set_index('Num_Acc'), on='Num_Acc')
df = df.join(df3.set_index('Accident_Id'), on='Num_Acc', validate='m:1')
df = df.join(df4.set_index('Num_Acc'), on='Num_Acc', lsuffix='_')

/tmp/ipykernel_1252/1306549260.py:5: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("dataset/lieux-2022.csv", sep=';')


In [3]:
# Suppression colonnes inutiles
df = df.drop(columns=['voie', 'v1', 'v2', 'pr', 'pr1', 'lartpc', 'larrout'
                      , 'num_veh', 'occutc', 'adr', 'senc','etatp','actp', 'manv', 'jour', 'dep', 'com', 'hrmn'
                      , 'lat', 'long', 'nbv'])

df = df.drop_duplicates(subset=['id_usager']) # retire les doublons dans les usagers

# Remplacement des valeurs NaN
df['an_nais'] = df['an_nais'].fillna(df['an_nais'].mode()[0])

# Convertir en entier
df['id_vehicule'] = df['id_vehicule'].apply(lambda l: l[0:3] + l[4:7])
df['id_vehicule'] = df['id_vehicule'].astype(int)
df['sexe'] = df['sexe'].astype(int)
df

,Num_Acc,id_usager,id_vehicule,place,catu,grav,sexe,an_nais,trajet,secu1,...,lum,agg,int,atm,col,catv,obs,obsm,choc,motor
0,202200000001,1 099 700,813952,1,1,3,1,2008.0,5,2,...,1,2,3,1,3,2,0,2,1,1
1,202200000001,1 099 701,813953,1,1,1,1,1948.0,5,1,...,1,2,3,1,3,2,0,2,1,1
2,202200000002,1 099 698,813950,1,1,4,1,1988.0,9,1,...,1,2,3,1,3,7,0,2,8,1
3,202200000002,1 099 699,813951,1,1,1,1,1970.0,4,1,...,1,2,3,1,3,7,0,2,8,1
4,202200000003,1 099 696,813948,1,1,1,1,2002.0,0,1,...,1,2,6,1,2,7,0,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126657,202200055301,968 230,715631,1,1,1,2,2002.0,5,1,...,1,1,3,1,3,7,0,0,8,1
126658,202200055301,968 231,715631,8,2,3,2,2004.0,5,1,...,1,1,3,1,3,7,0,0,8,1
126659,202200055301,968 232,715632,1,1,4,2,1953.0,5,1,...,1,1,3,1,3,7,0,0,8,1
126660,202200055302,968 228,715629,1,1,3,1,1992.0,1,2,...,1,2,1,1,2,33,0,2,1,1


In [4]:
for column in df.columns:
  if df[column].isnull().values.any() == True:
    print(column, df[column].isnull().values.any()) # afficher s'il y a des valeurs nulles

On peut déjà remarquer qu'il y a des valeurs nulles pour l'attribut *an_nais*.

In [5]:
# valeurs numériques
numerical_features = list(df.select_dtypes(include=np.number).columns)

# valeurs catégorielles
categorical_features = list(set(df.columns)-set(numerical_features))
print("numerical : ", numerical_features)
print("categorical : ", categorical_features)

numerical :  ['Num_Acc', 'id_vehicule', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'catr', 'circ', 'vosp', 'prof', 'plan', 'surf', 'infra', 'situ', 'vma', 'mois', 'an', 'lum', 'agg', 'int', 'atm', 'col', 'catv', 'obs', 'obsm', 'choc', 'motor']
categorical :  ['id_usager']


In [6]:
# Passage d'une caractéristique à un attribut
def to_attribute(id_valid, cat_a, cat_b):
    m = []
    for i in df['id_vehicule']:
        if i in id_valid['id_vehicule'].to_numpy():
            m.append(cat_a)
        else:
            m.append(cat_b)
    return m

In [7]:
df_2 = df.copy()
# On crée un attribut pour les accidents mortels

# Véhicule impliqué dans un accident mortel
with_death = df[df['grav'] == 2]
acc_with_death = with_death['Num_Acc']

d = []
for i in df['Num_Acc']:
    if i in acc_with_death.to_numpy():
        d.append(1)
    else:
        d.append(0)
df_2['mortal'] = d


# Accident impliquant un piéton
has_pedestrian = df[df['catu'] == 3]
# Sexe du conducteur
driver = df[(df['catu'] == 1) & (df['sexe'] == 1)]

p = to_attribute(has_pedestrian, 1, 0)
dr = to_attribute(driver, 1, 0)

df_2['pieton'] = p
df_2['sexe_conducteur'] = dr
df_2 = df_2.drop_duplicates(subset=['id_vehicule'])

In [8]:
# 1 : Bicyclette
# 2 : cyclomoteur
# 3 : VL (voiture)
# 4 : Utilitaire
# 5 : Motocyclette
# 0 : autre

# On réduit les carégories de véhicules
cat = [1, 2, 7, 33, 10]
corresp = {1:1, 2:2, 7:3, 10:4, 33:5}

r = []
for i in df_2['catv']:
    if i in cat:
        r.append(corresp[i])
    else:
        r.append(0)
df_2['catv'] = r


In [9]:
# On enlève la catégorie peu repésentées qu'on ajoute dans une catégorie autre (identifiant 5)

r = []
for i in df_2['catr']:
    if i > 4:
        r.append(5)
    else:
        r.append(i)
df_2['catr'] = r

In [10]:
# Découpage en 3 catégories de vitesse
# Identifiants:
#   1 -> <50
#   2 -> >=50 && < 100
#   3 -> >=100

v = df_2['vma']
cat = []
for i in v:
    if i < 50:
        cat.append(1)
    elif i >= 100:
        cat.append(3)
    else:
        cat.append(2)
df_2['vma'] = cat 

In [11]:
# Age du conducteur du véhicule
driver_age = df_2[(df_2['catu'] == 1)][['an_nais', 'id_vehicule', 'an']]
driver_age['an_nais'] = driver_age['an'] - driver_age['an_nais']

df_2 = df_2.join(driver_age.set_index('id_vehicule'), on='id_vehicule', lsuffix='_')
df_2 = df_2.rename(columns={'an_nais':'age'})

# On enlève les attributs qui ne sont plus utiles
df_2 = df_2.drop(columns=['an_nais_','grav', 'sexe','catu', 'Num_Acc', 'id_usager', 'id_vehicule',
                          'secu1','secu2','secu3','an_', 'an'])

In [12]:
for column in df_2.columns:
  if df_2[column].isnull().values.any() == True:
    print(column, df_2[column].isnull().values.any()) # afficher s'il y a des valeurs nulles


age True


In [13]:
df_2['age'] = df_2['age'].fillna(df_2['age'].mode()[0])


In [14]:
df_2.columns

Index(['place', 'trajet', 'locp', 'catr', 'circ', 'vosp', 'prof', 'plan',
       'surf', 'infra', 'situ', 'vma', 'mois', 'lum', 'agg', 'int', 'atm',
       'col', 'catv', 'obs', 'obsm', 'choc', 'motor', 'mortal', 'pieton',
       'sexe_conducteur', 'age'],
      dtype='object')

In [15]:
df_2.dtypes

place                int64
trajet               int64
locp                 int64
catr                 int64
circ                 int64
vosp                 int64
prof                 int64
plan                 int64
surf                 int64
infra                int64
situ                 int64
vma                  int64
mois                 int64
lum                  int64
agg                  int64
int                  int64
atm                  int64
col                  int64
catv                 int64
obs                  int64
obsm                 int64
choc                 int64
motor                int64
mortal               int64
pieton               int64
sexe_conducteur      int64
age                float64
dtype: object

In [ ]:
val = [len(df_2[df_2.mortal == 1]), len(df_2[df_2.mortal == 0])]
labels = ['Accident mortel', 'Accident non mortel']

px.pie(values=val, names=labels)

In [ ]:
val = [len(df_2[df_2.pieton == 1]), len(df_2[df_2.pieton == 0])]
labels = ['Implique piéton', 'N\'implique pas de piéton']

px.pie(values=val, names=labels)

In [ ]:
fig = px.histogram(df_2, x="catv")
fig.show()

In [ ]:
fig = px.box(df_2, x="age")
fig.show()

In [ ]:
fig = px.histogram(df_2, x="sexe_conducteur")
fig.show()

In [ ]:
fig = px.histogram(df_2, x="motor")
fig.show()

In [ ]:
fig = px.histogram(df_2, x="catr")
fig.show()

In [ ]:
fig = px.histogram(df_2, x="col")
fig.show()

In [42]:
def rapport_corr(x, y):
  '''
  Calcule le rapport de correlation entre une variable
  qualitative (x) et une variable quantitative (y)
  x : list of the qualitative variable observations
  y : list of the quantitative variable obesvations
  x and y must be of the same  length
  '''
  if len(x)!=len(y):
    print("The two list doesn't have the same length {len(x)}!={len(y)}")
    return -1
  mean_y = np.mean(y)
  se2 = 0
  sr2 = 0
  for cat in set(x):
    y_cat = [y[i] for i in range(len(x)) if x[i]==cat]
    se2 += len(y_cat)*(np.mean(y_cat)-mean_y)**2
    sr2 += len(y_cat)*np.var(y_cat)
  se2/=len(y)
  sr2/=len(y)
  return se2/(se2 + sr2)

In [43]:
def analyse_bi_quali_quanti(quali, quanti, df):
  # Rapport de correlation
  rapp = rapport_corr(df[quali].values, df[quanti].values)
  print(f"Rapport de corr {quali} X {quanti}: {rapp} ")
  # boite a moustaches
  bam = px.box(df, x=quali, y=quanti)
  bam.show()
  # histogramme
  hist = px.histogram(df, x=quanti, color=quali, barmode='overlay', opacity=0.75)
  hist.show()

  hist_prop = px.histogram(df, x=quanti, color=quali, barmode='overlay', opacity=0.75, histnorm="probability")
  hist_prop.show()

In [ ]:
features = ['age', 'mois', 'catr', 'mortal', 'sexe_conducteur']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("catv", exp, df_2)

In [ ]:
features = ['age', 'mois', 'catr', 'mortal']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("vma", exp, df_2)

In [ ]:
features = ['age', 'vma', 'mois', 'catr', 'mortal']

for exp in features:
    print(exp)
    analyse_bi_quali_quanti("col", exp, df_2)

In [37]:
df_3 = df_2.drop(columns=['mortal'])
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_3, df_2[label], test_size=0.33, random_state=42)

In [38]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=42)
clf = clf.fit(X_train, y_train)

preds = clf.predict(X_test)

clf.score(X_test, y_test)

0.9180322611679441

In [39]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, preds).ravel()
tn, fp, fn, tp

(28113, 1353, 1203, 514)

In [40]:
import plotly.express as px


fig = px.imshow([[tn, fp], [fn, tp]], text_auto=True, labels=dict(y="Truth", x="Pred"),
                x=["False", "True"],
                y=["False", "True"]
               )
fig.show()